In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# default_exp fe

In [1]:
from gensim.models import Word2Vec

In [4]:
import time

import gensim

class EpochSaver(gensim.models.callbacks.CallbackAny2Vec):
    """
    用于保存模型, 打印损失函数等等
    """
    def __init__(self, savedir='./', save_name='word2vector.model'):

        self.save_path = savedir+save_name

        self.epoch = 0

        self.pre_loss = 0

#         self.best_loss = 999999999.9
        self.best_loss = 1841668.00

        self.since = time.time()

    def on_epoch_end(self, model):
        
        self.epoch += 1
        cum_loss = model.get_latest_training_loss() # 返回的是从第一个epoch累计的
        if self.epoch % 1 == 0:

            epoch_loss = cum_loss - self.pre_loss

            time_taken = time.time() - self.since

            print('Epoch %d, loss: %.2f, time: %dmin %ds' %

                        (self.epoch, epoch_loss, time_taken//60, time_taken%60))

            if self.best_loss > epoch_loss:

                self.best_loss = epoch_loss

                print('Better model.Best loss: %.2f' % self.best_loss)

                model.save(self.save_path)

                print('Model %s save done!' % self.save_path)

        self.pre_loss = cum_loss

        self.since = time.time()

In [5]:
model = Word2Vec.load(os.path.join(args.DATA_DIR, 'data_gen/word2vector.model'))

In [6]:
model.wv.save_word2vec_format(os.path.join(args.DATA_DIR, 'data_gen/word2vector.model.bin'), binary=True)

In [9]:
from gensim.models import KeyedVectors
VOCAB_FILE = os.path.join(args.DATA_DIR, 'data_gen/word2vector.model.bin')
wv_from_text = KeyedVectors.load_word2vec_format(VOCAB_FILE, binary=True) 

In [11]:
type(wv_from_text.vectors)

numpy.ndarray

In [10]:
wv_from_text.vectors.shape

(3412773, 128)

In [12]:
wvs = pd.Series(wv_from_text.vectors.tolist(), index=wv_from_text.index2word)

In [13]:
wvs.head()

<e>        [1.1942813396453857, -1.1226128339767456, -0.8...
108133     [2.074798345565796, -2.4218647480010986, -0.27...
37298      [1.0948036909103394, -3.0849058628082275, -1.6...
304118     [2.1751441955566406, -3.6619222164154053, -1.7...
1837000    [-0.5107254981994629, -2.399601697921753, -1.3...
dtype: object

In [18]:
wvs.head()

<e>        [1.1942813396453857, -1.1226128339767456, -0.8...
108133     [2.074798345565796, -2.4218647480010986, -0.27...
37298      [1.0948036909103394, -3.0849058628082275, -1.6...
304118     [2.1751441955566406, -3.6619222164154053, -1.7...
1837000    [-0.5107254981994629, -2.399601697921753, -1.3...
dtype: object

In [16]:
wvs = wvs.map(np.array)

In [20]:
wvsf = wvs.to_frame()

In [25]:
wvsf.columns = ['creative_id_vec']

In [26]:
wvsf.head()

,creative_id_vec
<e>,"[1.1942813396453857, -1.1226128339767456, -0.8..."
108133,"[2.074798345565796, -2.4218647480010986, -0.27..."
37298,"[1.0948036909103394, -3.0849058628082275, -1.6..."
304118,"[2.1751441955566406, -3.6619222164154053, -1.7..."
1837000,"[-0.5107254981994629, -2.399601697921753, -1.3..."


# lib导入

In [1]:
# export
import gc
import tqdm
import os
from tx.config import * 
from tx.eda import * 
from loguru import logger
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行
# from multiprocessing.dummy import Pool
from sklearn.metrics import classification_report, accuracy_score
import lightgbm as lgb
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
trace = logger.add(os.path.join(args.DATA_DIR, 'data_gen/runtime.log'))  

# load data

In [3]:
user_train, click_log_train= get_trainset_raw()
ad_feat = get_train_ad_feat()

user_val, click_log_val = get_valset_raw()

In [4]:
ad_feat_test = get_test_ad_feat()

user_test, click_log_test = get_testset_raw()

user_test.shape, click_log_test.shape

((1000000, 0), (33585512, 4))

In [5]:
user_train.head(2)

,age,gender
user_id,,
1,4,1
2,10,1


In [32]:
user_train.gender.unique()

array(['1', '2'], dtype=object)

In [33]:
user_train.age.unique()

array(['4', '10', '5', '6', '9', '8', '7', '3', '2', '1'], dtype=object)

In [8]:
click_log_train.head(2)

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1


In [9]:
ad_feat.head(2)

,ad_id,product_id,product_category,advertiser_id,industry
creative_id,,,,,
1,1,\N,5,381,78
4,4,\N,5,108,202


In [5]:
user_train_val = user_train.append(user_val)

click_log_train_val = click_log_train.append(click_log_val)

click_log_train_val.shape

(30082771, 4)

In [6]:
click_log_train_val = pd.merge(click_log_train_val, ad_feat.reset_index(), on='creative_id')


(30082771, 9)

In [37]:
click_log_train_val.shape

(30082771, 9)

In [41]:
click_log_test = pd.merge(click_log_test, ad_feat_test.reset_index(), on='creative_id')
click_log_test.shape

(33585512, 9)

In [42]:
click_log_test.shape

(33585512, 9)

In [43]:
click_log_train_val = pd.merge(click_log_train_val, wvsf, left_on='creative_id', right_index=True)

In [44]:
def t(s):
   
    return (s.sum()/s.shape[0]).tolist()
    
dfg = click_log_train_val.groupby('user_id')['creative_id_vec'].agg(t)

In [45]:
dfg.head()

user_id
1        [1.3915553562916243, -0.7676784487871023, -0.7...
10       [1.2332237243652344, -1.586252085864544, -0.95...
100      [-1.1402276013142023, 0.9095006907979647, 0.34...
1000     [1.84404336810112, -1.2125309293717146, -1.346...
10000    [1.0924896852184947, -1.7014844752848148, -1.1...
Name: creative_id_vec, dtype: object

In [47]:
df_tr = pd.DataFrame()
for i in range(128):
    df_tr[f'v{i}'] = dfg.str.get(i)

In [48]:
df_tr.shape

(900000, 128)

In [46]:
click_log_test = pd.merge(click_log_test, wvsf, left_on='creative_id', right_index=True)
dfg_test = click_log_test.groupby('user_id')['creative_id_vec'].agg(t)
dfg_test.head()

user_id
3000001    [0.42857617884874344, -0.4139093106443232, 0.6...
3000002    [1.7154054950739517, -1.0109891583570507, -1.2...
3000003    [1.0108042544323124, -1.3299903501239088, -0.7...
3000004    [1.4207156091928481, -1.3549318890087307, -0.6...
3000005    [0.4734301108567576, -1.39425988694919, -0.837...
Name: creative_id_vec, dtype: object

In [49]:
df_te = pd.DataFrame()
for i in range(128):
    df_te[f'v{i}'] = dfg_test.str.get(i)
df_te.shape

(1000000, 128)

In [10]:
click_log_train_val['weekday'] = click_log_train_val.time % 7

In [11]:
click_log_test['weekday'] = click_log_test.time % 7

In [12]:
def aggregate_features(df_, feat, agg_func_dict, prefix=''):
    """
    一阶特征
    """
    df = df_.copy()
    logger.info(f'gen 特征 for {",".join(feat)}...')
    

    agg_df = df.groupby(feat).agg(agg_func_dict)
    print(agg_df.columns.values)
    agg_df.columns = [prefix + '_'.join(col).strip() for col in agg_df.columns.values]
    logger.info(f'gen 特征 for {",".join(feat)}...end')
    
    return agg_df

In [13]:
agg_func = {
        'time':  ['count','nunique'],  # 日志数，存在日志的天数
        'creative_id':  ['nunique'],  # 
        'weekday':  ['nunique'],
        'ad_id':  ['nunique'],
        'product_id':  ['nunique'],
        'product_category':  ['nunique'],
        'advertiser_id':  ['nunique'],
        'industry':  ['nunique'],
        'click_times':  ['sum','mean','median','max','min','std'],
        } 
agg_df1_test = aggregate_features(click_log_test, ['user_id'], agg_func)

2020-06-20 09:23:55.663 | INFO     | __main__:aggregate_features:6 - gen 特征 for user_id...
2020-06-20 09:31:45.572 | INFO     | __main__:aggregate_features:12 - gen 特征 for user_id...end


[('time', 'count') ('time', 'nunique') ('creative_id', 'nunique')
 ('weekday', 'nunique') ('ad_id', 'nunique') ('product_id', 'nunique')
 ('product_category', 'nunique') ('advertiser_id', 'nunique')
 ('industry', 'nunique') ('click_times', 'sum') ('click_times', 'mean')
 ('click_times', 'median') ('click_times', 'max') ('click_times', 'min')
 ('click_times', 'std')]


In [ ]:
agg_df1 = aggregate_features(click_log_train_val, ['user_id'], agg_func)

In [27]:
agg_df1.head(2)

,time_count,time_nunique,creative_id_nunique,weekday_nunique,ad_id_nunique,product_id_nunique,product_category_nunique,advertiser_id_nunique,industry_nunique,click_times_sum,click_times_mean,click_times_median,click_times_max,click_times_min,click_times_std
user_id,,,,,,,,,,,,,,,
1,13,10,12,5,12,6,3,12,9,14,1.076923,1.0,2,1,0.27735
10,10,10,9,5,9,5,3,8,5,10,1.000000,1.0,1,1,0.00000


In [ ]:
user_train_val = pd.merge(user_train_val, agg_df1, left_index=True, right_index=True)
user_test = pd.merge(user_test, agg_df1_test, left_index=True, right_index=True)

In [ ]:
agg_func = {
        'creative_id':  ['count','nunique'],  # 
        'ad_id':  ['nunique'],
        'product_id':  ['nunique'],
        'product_category':  ['nunique'],
        'advertiser_id':  ['nunique'],
        'industry':  ['nunique'],
        'click_times':  ['sum','mean','median','max','min','std'],
        } 
agg_df_time_test = aggregate_features(click_log_test, ['user_id', 'time'], agg_func)

In [ ]:
agg_df_time = aggregate_features(click_log_train_val, ['user_id', 'time'], agg_func)

agg_df_time = agg_df_time.unstack()

agg_df_time.columns = [f'{i[0]}_time{i[1]}' for i in agg_df_time.columns]

agg_df_time = agg_df_time.fillna(0)

In [ ]:
agg_df_time_test = agg_df_time_test.unstack()

agg_df_time_test.columns = [f'{i[0]}_time{i[1]}' for i in agg_df_time_test.columns]

agg_df_time_test = agg_df_time_test.fillna(0)

In [ ]:
user_train_val = pd.merge(user_train_val, agg_df_time, left_index=True, right_index=True)
user_test = pd.merge(user_test, agg_df_time_test, left_index=True, right_index=True)

In [48]:
agg_df_time.head(2)

,creative_id_count_time1,creative_id_count_time2,creative_id_count_time3,creative_id_count_time4,creative_id_count_time5,creative_id_count_time6,creative_id_count_time7,creative_id_count_time8,creative_id_count_time9,creative_id_count_time10,creative_id_count_time11,creative_id_count_time12,creative_id_count_time13,creative_id_count_time14,creative_id_count_time15,creative_id_count_time16,creative_id_count_time17,creative_id_count_time18,creative_id_count_time19,creative_id_count_time20,creative_id_count_time21,creative_id_count_time22,creative_id_count_time23,creative_id_count_time24,creative_id_count_time25,creative_id_count_time26,creative_id_count_time27,creative_id_count_time28,creative_id_count_time29,creative_id_count_time30,creative_id_count_time31,creative_id_count_time32,creative_id_count_time33,creative_id_count_time34,creative_id_count_time35,creative_id_count_time36,creative_id_count_time37,creative_id_count_time38,creative_id_count_time39,creative_id_count_time40,creative_id_count_time41,creative_id_count_time42,creative_id_count_time43,creative_id_count_time44,creative_id_count_time45,creative_id_count_time46,creative_id_count_time47,creative_id_count_time48,creative_id_count_time49,creative_id_count_time50,...,click_times_std_time42,click_times_std_time43,click_times_std_time44,click_times_std_time45,click_times_std_time46,click_times_std_time47,click_times_std_time48,click_times_std_time49,click_times_std_time50,click_times_std_time51,click_times_std_time52,click_times_std_time53,click_times_std_time54,click_times_std_time55,click_times_std_time56,click_times_std_time57,click_times_std_time58,click_times_std_time59,click_times_std_time60,click_times_std_time61,click_times_std_time62,click_times_std_time63,click_times_std_time64,click_times_std_time65,click_times_std_time66,click_times_std_time67,click_times_std_time68,click_times_std_time69,click_times_std_time70,click_times_std_time71,click_times_std_time72,click_times_std_time73,click_times_std_time74,click_times_std_time75,click_times_std_time76,click_times_std_time77,click_times_std_time78,click_times_std_time79,click_times_std_time80,click_times_std_time81,click_times_std_time82,click_times_std_time83,click_times_std_time84,click_times_std_time85,click_times_std_time86,click_times_std_time87,click_times_std_time88,click_times_std_time89,click_times_std_time90,click_times_std_time91
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
agg_func = {
        'creative_id':  ['count','nunique'],  # 
        'ad_id':  ['nunique'],
        'product_id':  ['nunique'],
        'product_category':  ['nunique'],
        'advertiser_id':  ['nunique'],
        'industry':  ['nunique'],
        'click_times':  ['sum','mean','median','max','min','std'],
        } 
agg_df_weekday_test = aggregate_features(click_log_test, ['user_id', 'weekday'], agg_func)

In [53]:
agg_df_weekday.head(2)

creative_id_count  creative_id_nunique  ad_id_nunique  product_id_nunique  product_category_nunique  advertiser_id_nunique  industry_nunique  click_times_sum  click_times_mean  click_times_median  click_times_max  click_times_min  click_times_std
user_id weekday                                                                                                                                                                                                                                                        
1       1                        3                    3              3                   3                         2                      3                 3                3               1.0                 1.0                1                1              0.0
        3                        2                    2              2                   2                         2                      2                 2                2               1.0                 1.0                1                1              0.0

In [ ]:
agg_df_weekday = aggregate_features(click_log_train_val, ['user_id', 'weekday'], agg_func)
agg_df_weekday = agg_df_weekday.unstack()

agg_df_weekday.columns = [f'{i[0]}_weekday{i[1]}' for i in agg_df_weekday.columns]

agg_df_weekday = agg_df_weekday.fillna(0)
user_train_val = pd.merge(user_train_val, agg_df_weekday, left_index=True, right_index=True)

In [55]:
user_train_val.head(2)

,age,gender,time_count,time_nunique,creative_id_nunique,weekday_nunique,ad_id_nunique,product_id_nunique,product_category_nunique,advertiser_id_nunique,industry_nunique,click_times_sum,click_times_mean,click_times_median,click_times_max,click_times_min,click_times_std,creative_id_count_time1,creative_id_count_time2,creative_id_count_time3,creative_id_count_time4,creative_id_count_time5,creative_id_count_time6,creative_id_count_time7,creative_id_count_time8,creative_id_count_time9,creative_id_count_time10,creative_id_count_time11,creative_id_count_time12,creative_id_count_time13,creative_id_count_time14,creative_id_count_time15,creative_id_count_time16,creative_id_count_time17,creative_id_count_time18,creative_id_count_time19,creative_id_count_time20,creative_id_count_time21,creative_id_count_time22,creative_id_count_time23,creative_id_count_time24,creative_id_count_time25,creative_id_count_time26,creative_id_count_time27,creative_id_count_time28,creative_id_count_time29,creative_id_count_time30,creative_id_count_time31,creative_id_count_time32,creative_id_count_time33,...,advertiser_id_nunique_weekday6,industry_nunique_weekday0,industry_nunique_weekday1,industry_nunique_weekday2,industry_nunique_weekday3,industry_nunique_weekday4,industry_nunique_weekday5,industry_nunique_weekday6,click_times_sum_weekday0,click_times_sum_weekday1,click_times_sum_weekday2,click_times_sum_weekday3,click_times_sum_weekday4,click_times_sum_weekday5,click_times_sum_weekday6,click_times_mean_weekday0,click_times_mean_weekday1,click_times_mean_weekday2,click_times_mean_weekday3,click_times_mean_weekday4,click_times_mean_weekday5,click_times_mean_weekday6,click_times_median_weekday0,click_times_median_weekday1,click_times_median_weekday2,click_times_median_weekday3,click_times_median_weekday4,click_times_median_weekday5,click_times_median_weekday6,click_times_max_weekday0,click_times_max_weekday1,click_times_max_weekday2,click_times_max_weekday3,click_times_max_weekday4,click_times_max_weekday5,click_times_max_weekday6,click_times_min_weekday0,click_times_min_weekday1,click_times_min_weekday2,click_times_min_weekday3,click_times_min_weekday4,click_times_min_weekday5,click_times_min_weekday6,click_times_std_weekday0,click_times_std_weekday1,click_times_std_weekday2,click_times_std_weekday3,click_times_std_weekday4,click_times_std_weekday5,click_times_std_weekday6
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4,1,13,10,12,5,12,6,3,12,9,14,1.076923,1.0,2,1,0.277350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,3.0,0.0,2.0,3.0,1.0,3.0,0.0,3.0,0.0,2.0,3.0,1.0,5.0,0.0,1.0,0.0,1.000,1.0,1.0,1.25,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.5
2,10,1,45,28,42,7,42,20,3,36,15,46,1.022222,1.0,2,1,0.149071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,4.0,6.0,2.0,3.0,7.0,7.0,2.0,4.0,14.0,2.0,4.0,9.0,11.0,2.0,4.0,1.0,1.0,1.0,1.125,1.0,1.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.353553,0.0,0.0,0.0


In [ ]:

agg_df_weekday_test = agg_df_weekday_test.unstack()

agg_df_weekday_test.columns = [f'{i[0]}_weekday{i[1]}' for i in agg_df_weekday_test.columns]

agg_df_weekday_test = agg_df_weekday_test.fillna(0)
user_test = pd.merge(user_test, agg_df_weekday_test, left_index=True, right_index=True)

In [ ]:
agg_func = {
        'time':  ['count','nunique'],  # 日志数，存在日志的天数
        'creative_id':  ['nunique'],  # 
        'weekday':  ['nunique'],
        'ad_id':  ['nunique'],
        'product_id':  ['nunique'],
        'advertiser_id':  ['nunique'],
        'industry':  ['nunique'],
        'click_times':  ['sum','mean','median','max','min','std'],
        } 
agg_df_product_category_test = aggregate_features(click_log_test, ['user_id', 'product_category'], agg_func)

In [ ]:
agg_df_product_category = aggregate_features(click_log_train_val, ['user_id', 'product_category'], agg_func)

In [59]:
agg_df_product_category.head(2)

time_count  time_nunique  creative_id_nunique  weekday_nunique  ad_id_nunique  product_id_nunique  advertiser_id_nunique  industry_nunique  click_times_sum  click_times_mean  click_times_median  click_times_max  click_times_min  click_times_std
user_id product_category                                                                                                                                                                                                                                                      
1       18                         2             2                    2                2              2                   1                      2                 2                2          1.000000                 1.0                1                1         0.000000
        2                          6             5                    5                4              5                   5                      5                 4                7          1.166667                 1.0                2                1         0.408248

In [ ]:
agg_df_product_category = agg_df_product_category.unstack()

agg_df_product_category.columns = [f'{i[0]}_product_category{i[1]}' for i in agg_df_product_category.columns]

agg_df_product_category = agg_df_product_category.fillna(0)
user_train_val = pd.merge(user_train_val, agg_df_product_category, left_index=True, right_index=True)

In [ ]:
agg_df_product_category_test = agg_df_product_category_test.unstack()

agg_df_product_category_test.columns = [f'{i[0]}_product_category{i[1]}' for i in agg_df_product_category_test.columns]

agg_df_product_category_test = agg_df_product_category_test.fillna(0)
user_test = pd.merge(user_test, agg_df_product_category_test, left_index=True, right_index=True)

In [70]:
# agg_func = {
#         'time':  ['count','nunique'],  # 日志数，存在日志的天数
#         'creative_id':  ['nunique'],  # 
#         'weekday':  ['nunique'],
#         'ad_id':  ['nunique'],
#         'product_id':  ['nunique'],
#         'product_category':  ['nunique'],
#         'advertiser_id':  ['nunique'],
#         'click_times':  ['sum','mean','median','max','min','std'],
#         } 
# agg_df_industry_test = aggregate_features(click_log_test, ['user_id', 'industry'], agg_func)

# agg_df_industry = aggregate_features(click_log_train_val, ['user_id', 'industry'], agg_func)

# agg_df_industry.head(2)

# agg_df_industry = agg_df_industry.unstack()

# agg_df_industry.columns = [f'{i[0]}_industry{i[1]}' for i in agg_df_industry.columns]

# agg_df_industry = agg_df_industry.fillna(0)
# user_train_val = pd.merge(user_train_val, agg_df_industry, left_index=True, right_index=True)

# gc.collect()

# del agg_df_industry_test, agg_df_industry

# agg_df_industry_test = agg_df_industry_test.unstack()

# agg_df_industry_test.columns = [f'{i[0]}_industry{i[1]}' for i in agg_df_industry_test.columns]

# agg_df_industry_test = agg_df_industry_test.fillna(0)
# user_test = pd.merge(user_test, agg_df_industry_test, left_index=True, right_index=True)

2020-06-08 16:20:26.785 | INFO     | __main__:aggregate_features:6 - gen 特征 for user_id,industry...
2020-06-08 16:28:17.330 | INFO     | __main__:aggregate_features:12 - gen 特征 for user_id,industry...end


[('time', 'count') ('time', 'nunique') ('creative_id', 'nunique')
 ('weekday', 'nunique') ('ad_id', 'nunique') ('product_id', 'nunique')
 ('product_category', 'nunique') ('advertiser_id', 'nunique')
 ('click_times', 'sum') ('click_times', 'mean') ('click_times', 'median')
 ('click_times', 'max') ('click_times', 'min') ('click_times', 'std')]


In [75]:
user_test.shape

(1000000, 1541)

In [ ]:
user_test.to_pickle(os.path.join(args.DATA_DIR, 'data_gen/user_test_feat0608.pkl'))

In [84]:
from scipy import sparse

In [86]:
user_train_val.head(2)

,age,gender,time_count,time_nunique,creative_id_nunique,weekday_nunique,ad_id_nunique,product_id_nunique,product_category_nunique,advertiser_id_nunique,industry_nunique,click_times_sum,click_times_mean,click_times_median,click_times_max,click_times_min,click_times_std,creative_id_count_time1,creative_id_count_time2,creative_id_count_time3,creative_id_count_time4,creative_id_count_time5,creative_id_count_time6,creative_id_count_time7,creative_id_count_time8,creative_id_count_time9,creative_id_count_time10,creative_id_count_time11,creative_id_count_time12,creative_id_count_time13,creative_id_count_time14,creative_id_count_time15,creative_id_count_time16,creative_id_count_time17,creative_id_count_time18,creative_id_count_time19,creative_id_count_time20,creative_id_count_time21,creative_id_count_time22,creative_id_count_time23,creative_id_count_time24,creative_id_count_time25,creative_id_count_time26,creative_id_count_time27,creative_id_count_time28,creative_id_count_time29,creative_id_count_time30,creative_id_count_time31,creative_id_count_time32,creative_id_count_time33,...,click_times_std_industry50,click_times_std_industry51,click_times_std_industry52,click_times_std_industry53,click_times_std_industry54,click_times_std_industry55,click_times_std_industry56,click_times_std_industry57,click_times_std_industry58,click_times_std_industry59,click_times_std_industry6,click_times_std_industry60,click_times_std_industry61,click_times_std_industry62,click_times_std_industry63,click_times_std_industry64,click_times_std_industry65,click_times_std_industry66,click_times_std_industry67,click_times_std_industry68,click_times_std_industry69,click_times_std_industry70,click_times_std_industry71,click_times_std_industry72,click_times_std_industry73,click_times_std_industry74,click_times_std_industry75,click_times_std_industry76,click_times_std_industry78,click_times_std_industry79,click_times_std_industry8,click_times_std_industry81,click_times_std_industry82,click_times_std_industry83,click_times_std_industry84,click_times_std_industry85,click_times_std_industry86,click_times_std_industry87,click_times_std_industry88,click_times_std_industry89,click_times_std_industry9,click_times_std_industry90,click_times_std_industry92,click_times_std_industry93,click_times_std_industry94,click_times_std_industry96,click_times_std_industry97,click_times_std_industry98,click_times_std_industry99,click_times_std_industry\N
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4,1,13,10,12,5,12,6,3,12,9,14,1.076923,1.0,2,1,0.277350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10,1,45,28,42,7,42,20,3,36,15,46,1.022222,1.0,2,1,0.149071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
user_train_val.columns

Index(['age', 'gender', 'time_count', 'time_nunique', 'creative_id_nunique',
       'weekday_nunique', 'ad_id_nunique', 'product_id_nunique',
       'product_category_nunique', 'advertiser_id_nunique',
       ...
       'click_times_std_industry9', 'click_times_std_industry90',
       'click_times_std_industry92', 'click_times_std_industry93',
       'click_times_std_industry94', 'click_times_std_industry96',
       'click_times_std_industry97', 'click_times_std_industry98',
       'click_times_std_industry99', 'click_times_std_industry\N'],
      dtype='object', length=6107)

In [100]:
gc.collect()

108

In [101]:
user_train_val[['age', 'gender']+user_test.columns.tolist()].to_pickle(os.path.join(args.DATA_DIR, 'data_gen/user_train_val_feat0608.pkl'))

In [107]:
user_train_val = user_train_val[['age', 'gender']+user_test.columns.tolist()]

In [90]:

user_train_val.to_pickle(os.path.join(args.DATA_DIR, 'data_gen/user_train_val_feat0608.pkl'))

MemoryError: 

In [51]:
user_train_val = pd.read_pickle(os.path.join(args.DATA_DIR, 'data_gen/user_train_val_feat0608.pkl'))
user_test = pd.read_pickle(os.path.join(args.DATA_DIR, 'data_gen/user_test_feat0608.pkl'))

In [52]:
df_tr.to_pickle(os.path.join(args.DATA_DIR, 'data_gen/df_tr.pkl'))
df_te.to_pickle(os.path.join(args.DATA_DIR, 'data_gen/df_te.pkl'))

In [53]:
user_train_val.shape

(900000, 1543)

In [54]:
user_test.shape

(1000000, 1541)

In [55]:
user_test.head()

,time_count,time_nunique,creative_id_nunique,weekday_nunique,ad_id_nunique,product_id_nunique,product_category_nunique,advertiser_id_nunique,industry_nunique,click_times_sum,click_times_mean,click_times_median,click_times_max,click_times_min,click_times_std,creative_id_count_time1,creative_id_count_time2,creative_id_count_time3,creative_id_count_time4,creative_id_count_time5,creative_id_count_time6,creative_id_count_time7,creative_id_count_time8,creative_id_count_time9,creative_id_count_time10,creative_id_count_time11,creative_id_count_time12,creative_id_count_time13,creative_id_count_time14,creative_id_count_time15,creative_id_count_time16,creative_id_count_time17,creative_id_count_time18,creative_id_count_time19,creative_id_count_time20,creative_id_count_time21,creative_id_count_time22,creative_id_count_time23,creative_id_count_time24,creative_id_count_time25,creative_id_count_time26,creative_id_count_time27,creative_id_count_time28,creative_id_count_time29,creative_id_count_time30,creative_id_count_time31,creative_id_count_time32,creative_id_count_time33,creative_id_count_time34,creative_id_count_time35,...,click_times_max_product_category13,click_times_max_product_category14,click_times_max_product_category15,click_times_max_product_category16,click_times_max_product_category17,click_times_max_product_category18,click_times_max_product_category2,click_times_max_product_category3,click_times_max_product_category4,click_times_max_product_category5,click_times_max_product_category6,click_times_max_product_category7,click_times_max_product_category8,click_times_max_product_category9,click_times_min_product_category1,click_times_min_product_category10,click_times_min_product_category11,click_times_min_product_category12,click_times_min_product_category13,click_times_min_product_category14,click_times_min_product_category15,click_times_min_product_category16,click_times_min_product_category17,click_times_min_product_category18,click_times_min_product_category2,click_times_min_product_category3,click_times_min_product_category4,click_times_min_product_category5,click_times_min_product_category6,click_times_min_product_category7,click_times_min_product_category8,click_times_min_product_category9,click_times_std_product_category1,click_times_std_product_category10,click_times_std_product_category11,click_times_std_product_category12,click_times_std_product_category13,click_times_std_product_category14,click_times_std_product_category15,click_times_std_product_category16,click_times_std_product_category17,click_times_std_product_category18,click_times_std_product_category2,click_times_std_product_category3,click_times_std_product_category4,click_times_std_product_category5,click_times_std_product_category6,click_times_std_product_category7,click_times_std_product_category8,click_times_std_product_category9
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3131989,29,13,25,6,25,10,4,19,11,29,1.000000,1.0,1,1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,2.0,1.0,0.0,5.0,2.0,0.0,0.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3131990,41,20,37,7,36,8,3,29,12,50,1.219512,1.0,3,1,0.524985,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.631085,0.383482,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3131994,13,10,13,7,13,12,2,13,5,15,1.153846,1.0,2,1,0.375534,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0

In [57]:
user_train_val = pd.merge(user_train_val, df_tr, left_index=True, right_index=True)

In [58]:
user_test = pd.merge(user_test, df_te, left_index=True, right_index=True)

In [59]:
user_train_val.shape

(900000, 1671)

In [60]:
user_test.shape

(1000000, 1669)

# get_target_features

In [61]:
from sklearn.model_selection import KFold
folds = KFold(n_splits=5, shuffle=True, random_state=2020)

In [83]:
click_log_train_val['weekday'] = click_log_train_val.time % 7

click_log_test['weekday'] = click_log_test.time % 7

In [84]:
click_log_train_val.head(2)

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,creative_id_vec,weekday
0,9,30920,567330,1,504423,30673,3,32638,319,"[-2.299884557723999, 0.29670771956443787, 0.07...",2
1,15,320815,567330,1,504423,30673,3,32638,319,"[-2.299884557723999, 0.29670771956443787, 0.07...",1


In [85]:
# def get_target_features(df_, user_, df_test_, user_test_):

df_ = click_log_train_val
user = user_train_val
df_test_ = click_log_test
user_test_ = user_test

user['age'] = user.age.map(int)
user['gender'] = user.gender.map(int)



In [86]:
ages = pd.get_dummies(user['age'])

ages.columns = [f'age{i}' for i in ages.columns]

user = pd.concat([user, ages], axis=1)

In [87]:
pd.get_dummies(user.iloc[:5]['age'])

,4,5,6,10
user_id,,,,
1,1,0,0,0
2,0,0,0,1
4,0,1,0,0
6,0,0,1,0
7,0,0,1,0


In [88]:
user.head(2)

,age,gender,time_count,time_nunique,creative_id_nunique,weekday_nunique,ad_id_nunique,product_id_nunique,product_category_nunique,advertiser_id_nunique,industry_nunique,click_times_sum,click_times_mean,click_times_median,click_times_max,click_times_min,click_times_std,creative_id_count_time1,creative_id_count_time2,creative_id_count_time3,creative_id_count_time4,creative_id_count_time5,creative_id_count_time6,creative_id_count_time7,creative_id_count_time8,creative_id_count_time9,creative_id_count_time10,creative_id_count_time11,creative_id_count_time12,creative_id_count_time13,creative_id_count_time14,creative_id_count_time15,creative_id_count_time16,creative_id_count_time17,creative_id_count_time18,creative_id_count_time19,creative_id_count_time20,creative_id_count_time21,creative_id_count_time22,creative_id_count_time23,creative_id_count_time24,creative_id_count_time25,creative_id_count_time26,creative_id_count_time27,creative_id_count_time28,creative_id_count_time29,creative_id_count_time30,creative_id_count_time31,creative_id_count_time32,creative_id_count_time33,...,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,age1,age2,age3,age4,age5,age6,age7,age8,age9,age10
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4,1,13,10,12,5,12,6,3,12,9,14,1.076923,1.0,2,1,0.277350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.915442,-0.548912,0.084594,0.294922,0.527737,-0.227382,-0.606779,-0.095198,1.303235,-0.847233,0.587217,0.156023,-0.021273,-0.396385,-0.449135,-0.340950,0.273681,-0.187321,-0.781695,1.179834,0.157130,1.162027,0.161782,0.489548,1.670120,0.202454,-0.624394,0.141042,-0.454791,0.004345,0.480884,-0.221163,0.116925,-0.484452,0.907479,0.021886,-0.266677,-0.690255,-0.043603,0.101629,0,0,0,1,0,0,0,0,0,0
2,10,1,45,28,42,7,42,20,3,36,15,46,1.022222,1.0,2,1,0.149071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,-0.298496,-1.232939,0.390383,0.243690,0.131795,0.278845,-0.100705,0.023061,0.551289,-0.392215,0.031247,0.889121,-0.236300,0.133896,0.107021,-0.625199,0.879635,0.047189,-0.582213,-0.054233,-0.458023,0.513448,0.163088,0.611212,0.611917,0.048387,-0.097446,-0.068465,-0.650924,-0.232294,0.916631,0.095132,-0.193287,-0.104265,-0.356548,0.465837,-0.619839,0.297817,-0.317052,0.219973,0,0,0,0,0,0,0,0,0,1


In [69]:
gc.collect()

17949

In [70]:
user_test.columns[:-30]

Index(['time_count', 'time_nunique', 'creative_id_nunique', 'weekday_nunique',
       'ad_id_nunique', 'product_id_nunique', 'product_category_nunique',
       'advertiser_id_nunique', 'industry_nunique', 'click_times_sum',
       ...
       'v88', 'v89', 'v90', 'v91', 'v92', 'v93', 'v94', 'v95', 'v96', 'v97'],
      dtype='object', length=1639)

In [71]:
user_test.columns

Index(['time_count', 'time_nunique', 'creative_id_nunique', 'weekday_nunique',
       'ad_id_nunique', 'product_id_nunique', 'product_category_nunique',
       'advertiser_id_nunique', 'industry_nunique', 'click_times_sum',
       ...
       'v118', 'v119', 'v120', 'v121', 'v122', 'v123', 'v124', 'v125', 'v126',
       'v127'],
      dtype='object', length=1669)

In [89]:
param = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': ['binary_logloss', 'accuracy'], 
    'num_leaves': 300,  
    'min_data_in_leaf': 500,  
    'learning_rate': 0.05,  
    'feature_fraction': 0.8,  
    'bagging_fraction': 0.8,  
    'bagging_freq': 5,  
    'lambda_l1': 0.4,  
    'lambda_l2': 0.5,  
    'min_gain_to_split': 0.2,  
    'verbose': -1,
    'num_threads':8,
}

num_class = 10
param1 = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',  
    'num_class': num_class,  
    'metric': 'multi_error',  
    'num_leaves': 300,  
    'min_data_in_leaf': 500,  
    'learning_rate': 0.05,  
    'feature_fraction': 0.8,  
    'bagging_fraction': 0.8,  
    'bagging_freq': 5,  
    'lambda_l1': 0.4,  
    'lambda_l2': 0.5,  
    'min_gain_to_split': 0.2,  
    'verbose': -1,
    'num_threads':16,
}


In [146]:
param = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': ['binary_logloss', 'accuracy'], 
    'num_leaves': 2**7-1,
    'min_data_in_leaf': 25, 
    'learning_rate': 0.1,  
    'feature_fraction': 0.6,
    'bagging_fraction': 0.9,
    'bagging_seed': 11,
    'seed':1024,
    'lambda_l1': 0.2,  
    'verbose': -1,
    'num_threads':16,
}

num_class = 10
param1 = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',  
    'num_class': num_class,  
    'metric': 'multi_error',  
    'num_leaves': 2**7-1,
    'min_data_in_leaf': 25, 
    'learning_rate': 0.1,  
    'feature_fraction': 0.6,
    'bagging_fraction': 0.9,
    'bagging_seed': 11,
    'seed':1024,
    'lambda_l1': 0.2,  
    'verbose': -1,
    'num_threads':16,
}


In [93]:
targets = ['gender']+ages.columns.tolist()
# id_cols = 'time weekday creative_id ad_id product_id product_category advertiser_id industry'.split()
id_cols = 'time ad_id product_category advertiser_id'.split()

# train

In [148]:
predictions = np.zeros([len(user_test_)])

predictions1 = np.zeros([len(user_test_), num_class])

for fold_,(trn_idx,val_idx) in enumerate(folds.split(user)):
#     if fold_ == 0: continue
    gc.collect()
    user_train = user.iloc[trn_idx].copy()
    user_val = user.iloc[val_idx]
    df_train = pd.merge(df_, user_train[targets].reset_index())
    df_val = pd.merge(df_, user_val[targets].reset_index())
    cols = user_train.columns.tolist()
    cols = list(set(cols)-set(ages.columns.tolist()))
    cols.remove('age')
    cols.remove('gender')
#     for idc in id_cols:
# #         break
#         for tg in targets:
#             coln = f'{idc}_{tg}_te'
#             print(coln)
#             for tc in ['mean']:
#                 order_label = df_train.groupby(idc)[tg].agg(tc)

#                 df_train.loc[:, coln] = df_train[idc].map(order_label, na_action='ignore')
#                 df_val.loc[:, coln] = df_val[idc].map(order_label, na_action='ignore')
#                 df_test_.loc[:, coln] = df_test_[idc].map(order_label, na_action='ignore')

#             user_train.loc[:, f'{coln}_mean'] = df_train.groupby('user_id')[coln].mean()
#             user_train.loc[:, f'{coln}_median'] = df_train.groupby('user_id')[coln].median()

#             user_val.loc[:, f'{coln}_mean'] = df_val.groupby('user_id')[coln].mean()
#             user_val.loc[:, f'{coln}_median'] = df_val.groupby('user_id')[coln].median()

#             user_test_.loc[:, f'{coln}_mean'] = df_test_.groupby('user_id')[coln].mean()
#             user_test_.loc[:, f'{coln}_median'] = df_test_.groupby('user_id')[coln].median()
            
#             cols.append(f'{coln}_mean')
#             cols.append(f'{coln}_median')
#             gc.collect()
#     del df_train, df_val

    gc.collect()
    print("fold n°{}".format(fold_+1))
#     print(trn_idx)
#     print(cols)
    X_train = user_train[cols]
    X_val = user_val[cols]
    y_train = user_train['gender'].map(lambda x: int(x)-1)
    y_val = user_val['gender'].map(lambda x: int(x)-1)
    
    
    y_train1 = user_train['age'].map(lambda x: int(x)-1)
    y_val1 = user_val['age'].map(lambda x: int(x)-1)
    
    trn_data = lgb.Dataset(user_train[cols], y_train)
    val_data = lgb.Dataset(user_val[cols], y_val)
    
    trn_data1 = lgb.Dataset(user_train[cols], y_train1)
    val_data1 = lgb.Dataset(user_val[cols], y_val1)

    num_round = 1000
    # callback
    def reset_metrics():
        def callback(env):
    #         lgb_eval_new = lgb.Dataset(X_test, y_test, reference=lgb_train)

            if env.iteration % 100 == 1:
                rs = pd.Series(env.model.predict(X_val).tolist()).map(round)
                print(f'{env.iteration} predict...{accuracy_score(y_val, rs)}')
        callback.before_iteration = False
        callback.order = 0
        return callback
    
    # callback
    def reset_metrics1():
        def callback(env):
            if env.iteration % 100 == 1:
                print(f'{env.iteration} predict...{accuracy_score(y_val1, np.argmax(env.model.predict(X_val), axis=1))}')
        callback.before_iteration = False
        callback.order = 0
        return callback
    
    clf = lgb.train(param, 
                    trn_data, 
                    num_round, 
                    valid_sets = [trn_data, val_data], 
                    verbose_eval = 100, 
                    early_stopping_rounds = 100,
                    callbacks=[reset_metrics()])
    rs = pd.Series(clf.predict(X_val).tolist()).map(round)
    print(f'{num_round} predict...{accuracy_score(y_val, rs.tolist())}')
    #oof[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)    
    predictions += clf.predict(user_test_[cols], num_iteration=clf.best_iteration) / folds.n_splits
    
    
    clf1 = lgb.train(param1, 
                    trn_data1, 
                    num_round, 
                    valid_sets = [trn_data1, val_data1], 
                    verbose_eval = 100, 
                    early_stopping_rounds = 100,
                    callbacks=[reset_metrics1()])
    print(f'{num_round} predict...{accuracy_score(y_val1, np.argmax(clf1.predict(X_val), axis=1))}')
    #oof[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)    
    predictions1 += clf1.predict(user_test_[cols], num_iteration=clf1.best_iteration) / folds.n_splits


fold n°1
Training until validation scores don't improve for 100 rounds
1 predict...0.6698
[100]	training's binary_logloss: 0.198429	valid_1's binary_logloss: 0.217886
101 predict...0.9170222222222222
[200]	training's binary_logloss: 0.167394	valid_1's binary_logloss: 0.208428
201 predict...0.9213722222222223
[300]	training's binary_logloss: 0.146362	valid_1's binary_logloss: 0.205298
301 predict...0.9229333333333334
[400]	training's binary_logloss: 0.130734	valid_1's binary_logloss: 0.204558
401 predict...0.9236722222222222
[500]	training's binary_logloss: 0.118061	valid_1's binary_logloss: 0.204349
501 predict...0.9234833333333333
Early stopping, best iteration is:
[475]	training's binary_logloss: 0.120959	valid_1's binary_logloss: 0.204328
1000 predict...0.9234444444444444
Training until validation scores don't improve for 100 rounds
1 predict...0.22767777777777778
[100]	training's multi_error: 0.532792	valid_1's multi_error: 0.632722
101 predict...0.36741111111111113
[200]	training'

KeyboardInterrupt: 

In [110]:
# np.save(os.path.join(args.DATA_DIR, 'data_gen/rs_gender_arr_lgb_w2v.npy'), predictions)

# np.save(os.path.join(args.DATA_DIR, 'data_gen/rs_age_arr_lgb_w2v.npy'), predictions1)

## 纯统计特征
0.85和0.3

## 统计特征+target encoding
线下gender0.905，age0.33，线上1.25

In [ ]:
fold n°1
Training until validation scores don't improve for 100 rounds
1 predict...0.6698
[100]	training's binary_logloss: 0.0852306	valid_1's binary_logloss: 0.238045
101 predict...0.90935
Early stopping, best iteration is:
[70]	training's binary_logloss: 0.103788	valid_1's binary_logloss: 0.232108
1000 predict...0.9107388888888889
Training until validation scores don't improve for 100 rounds
1 predict...0.24253333333333332
[100]	training's multi_error: 0.0863069	valid_1's multi_error: 0.664633
101 predict...0.33511111111111114
Early stopping, best iteration is:
[41]	training's multi_error: 0.103697	valid_1's multi_error: 0.661639
1000 predict...0.3383611111111111

fold n°2
Training until validation scores don't improve for 100 rounds
1 predict...0.6699611111111111
[100]	training's binary_logloss: 0.085032	valid_1's binary_logloss: 0.241975
101 predict...0.9081166666666667
Early stopping, best iteration is:
[65]	training's binary_logloss: 0.108126	valid_1's binary_logloss: 0.235292
1000 predict...0.9093222222222223
Training until validation scores don't improve for 100 rounds
1 predict...0.23845
[100]	training's multi_error: 0.0866681	valid_1's multi_error: 0.665694
101 predict...0.33442777777777777
Early stopping, best iteration is:
[33]	training's multi_error: 0.106832	valid_1's multi_error: 0.662133
1000 predict...0.33786666666666665

## 统计特征+w2v
线下gender0.925，age0.0.3876，线上1.3184

## 单creativeid序列+HAN
线下0.94+0.44

线上1.288446 age0.369436 gender0.919

## transformer
0.936的0.404

线上1.342

In [105]:
rs = pd.Series(predictions).map(round)

In [162]:
rs.value_counts()

0    681737
1    318263
dtype: int64

In [106]:
rs.value_counts()

0    680598
1    319402
dtype: int64

In [107]:
user_test_['predicted_gender'] = rs.map(lambda x: str(int(x+1))).tolist()

In [108]:
user_test_['predicted_age'] = pd.Series(np.argmax(predictions1, axis=1).tolist()).map(lambda x: str(int(x+1))).tolist()

In [167]:
user_test_['predicted_gender'].value_counts()

1    681737
2    318263
Name: predicted_gender, dtype: int64

In [109]:
user_test_['predicted_age predicted_gender'.split()].reset_index().to_csv(os.path.join(args.DATA_DIR, 'data_gen/submission44.csv'), index=False)

# 融合

In [150]:
predictions = np.load(os.path.join(args.DATA_DIR, 'data_gen/rs_gender_arr_lgb_w2v.npy'))

predictions1 = np.load(os.path.join(args.DATA_DIR, 'data_gen/rs_age_arr_lgb_w2v.npy'))

In [113]:
rjq = pd.read_csv(os.path.join(args.DATA_DIR, 'data_gen/submission_jq.csv'))

In [115]:
rjq['user_id'] = rjq['user_id'].map(int).map(str)

In [117]:
rjq = rjq.set_index('user_id')

In [125]:
rjq.shape

(1000000, 12)

In [126]:
rjq = rjq.sort_index()

In [127]:
rjq.head()

,predicted_age1,predicted_age2,predicted_age3,predicted_age4,predicted_age5,predicted_age6,predicted_age7,predicted_age8,predicted_age9,predicted_age10,predicted_gender1,predicted_gender2
user_id,,,,,,,,,,,,
3000001,3.680792e-25,1.046126e-11,8.413182e-04,9.991441e-01,1.459543e-05,4.255965e-13,1.160107e-14,4.333467e-15,1.123161e-16,2.356512e-17,1.000000e+00,0.000000
3000002,8.753800e-41,4.290788e-12,2.674197e-14,2.903635e-20,5.280498e-12,2.250126e-05,9.645290e-01,3.544534e-02,3.226224e-06,4.509867e-19,3.266430e-36,1.000000
3000003,1.964534e-09,3.459816e-04,3.704170e-01,6.107929e-01,1.834657e-02,6.710326e-05,1.605230e-05,5.514042e-06,4.434732e-06,4.322189e-06,1.926374e-04,0.999807
3000004,2.281205e-12,8.892768e-05,9.134302e-01,8.628824e-02,1.921107e-04,2.924756e-07,4.591624e-08,9.326176e-08,1.505639e-07,3.437124e-08,9.998444e-01,0.000156
3000005,3.512691e-17,1.795604e-08,8.508657e-04,6.804658e-01,3.176074e-01,1.071157e-03,4.817906e-06,1.851300e-08,5.897118e-10,2.497647e-12,1.000000e+00,0.000000


In [162]:
dfr_age = pd.DataFrame(data=predictions1, index=user_test_.index).sort_index()

In [163]:
dfr_age.head()

,0,1,2,3,4,5,6,7,8,9
user_id,,,,,,,,,,
3000001,0.003779,0.287807,0.620248,0.073861,0.011196,0.002511,0.000488,0.000088,0.000016,0.000006
3000002,0.001094,0.002352,0.004941,0.022202,0.070706,0.159658,0.676122,0.058259,0.003876,0.000789
3000003,0.011418,0.121908,0.222840,0.155020,0.164470,0.164180,0.133176,0.019866,0.004233,0.002890
3000004,0.058376,0.499211,0.251999,0.093029,0.059176,0.024221,0.009172,0.004193,0.000589,0.000035
3000005,0.001503,0.068509,0.390635,0.290231,0.169490,0.057572,0.018674,0.002785,0.000563,0.000038


In [159]:
np.exp(0.404*8)

25.330266877941835

In [160]:
np.exp(0.387*8)

22.109336823384563

In [161]:
w1 = np.exp(0.404*8)
w2 = np.exp(0.387*8)

In [164]:
sub_age = rjq.iloc[:,:10].values * w1 + dfr_age.values*w2

In [165]:
dfr_gender = pd.DataFrame(data=predictions, index=user_test_.index).sort_index()

In [166]:
dfr_gender['a'] = 1 - dfr_gender.iloc[:,0]

In [167]:
dfr_gender.columns = ['c', 'd']
dfr_gender

,c,d
user_id,,
3000001,0.005240,0.994760
3000002,0.997738,0.002262
3000003,0.733715,0.266285
3000004,0.207463,0.792537
3000005,0.007629,0.992371
...,...,...
3999996,0.027396,0.972604
3999997,0.010741,0.989259
3999998,0.004914,0.995086


In [168]:
rjq.iloc[:, 11]

user_id
3000001    0.000000e+00
3000002    1.000000e+00
3000003    9.998074e-01
3000004    1.555733e-04
3000005    0.000000e+00
               ...     
3999996    0.000000e+00
3999997    6.097880e-38
3999998    0.000000e+00
3999999    0.000000e+00
4000000    0.000000e+00
Name: predicted_gender2, Length: 1000000, dtype: float64

In [169]:
sub_gender = rjq.iloc[:,10:].values * w1 + dfr_gender[['d', 'c']].values*w2

In [170]:
dfr_gender['predicted_gender'] = pd.Series(np.argmax(sub_gender, axis=1).tolist()).map(lambda x: str(int(x+1))).tolist()
dfr_gender['predicted_age'] = pd.Series(np.argmax(sub_age, axis=1).tolist()).map(lambda x: str(int(x+1))).tolist()
dfr_gender['predicted_age predicted_gender'.split()].reset_index().to_csv(os.path.join(args.DATA_DIR, 'data_gen/submission47.csv'), index=False)

In [171]:
dfr_gender.head()

,c,d,predicted_gender,predicted_age
user_id,,,,
3000001,0.005240,0.994760,1,4
3000002,0.997738,0.002262,2,7
3000003,0.733715,0.266285,2,4
3000004,0.207463,0.792537,1,3
3000005,0.007629,0.992371,1,4
